![ups logo](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png/640px-Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png)

# Inteligencia Artificial

# Practica 05

## Aplicaciones de Redes Neuronales y Redes Bayesianas como herramientas de soporte a la toma de decisiones. Despliegue de servicios de Inteligencia Artificial

### Autor: Diego Tapia

## Desarrollo

#### Esta practica se divide en siete partes:

1. Fase de preparación

2. Fase de modelado: Red Neuronal

3. Fase de modelado: Naive Bayes

4. **Comparacion de modelos**

5. **Fase de predicción de nuevos samples (Red Neuronal y Naive Bayes)**

6. Proyecto en Django Framework

7. Despliegue del proyecto


In [1]:
# Importacion de librerias
#%pip install scikeras
from time import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from matplotlib import*
from matplotlib.cm import register_cmap
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle
import seaborn as sns
import copy

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier #KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
#from keras.layers import Dropout

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from scipy import stats
import seaborn
from sklearn import set_config
print('Módulos importados')

Módulos importados


## Fase 4. Comparación de modelos

### Tabla 1. Comparacion de modelos; Red Nuronal, Naivey Nayes.

|Modelo     |Accuracy| Precision| Recall| F1-Score|
|-----------|---------|----------|-------|---------|
|Red Neuronal| 0.9723| 0.9260| 0.8378|0.8797|
|Naive Bayes| 0.8365| 0.3603| 0.4574|0.4031|

Para el modelo de Naive Bayes se utilizó el clasificador "Gaussian"


## Fase 5. Prediccion de nuevos samples

### Prediccion con Red Neuronal

Se carga el pipeline y la red neuronal

In [2]:
from keras.models import load_model
#Se carga el pipeline de transformación
def cargarPipeline(nombreArchivo):
    with open(nombreArchivo+'.pickle', 'rb') as handle:
        pipeline = pickle.load(handle)
    return pipeline

#Se carga el modelo entrenado
def cargarModelo(nombreArchivo):
    modelNN = load_model(nombreArchivo+".h5")
    return modelNN


In [3]:
#Esta es la función para calcular la certeza (confianza o probabilidad) asociada a la predicción de clase
def obtenerResultadosyCertezas(lista):
    predicciones=lista
    marcas=[]
    certezas=[]
    nuevomax=1
    nuevomin=0
    marca=-1
    certeza=-1
    for i in range(len(lista)):
        prediccion=lista[i]
        if (prediccion < 0.5):
            marca = 'No Acepta'
            maxa=0.5
            mina=0
            certeza=1-((prediccion-mina)/(maxa-mina)*(nuevomax-nuevomin)+nuevomin)
            certeza=str(int((certeza)*100))+'%'
        elif (prediccion >= 0.5):
            marca = 'Acepta'
            maxa=1
            mina=0.5
            certeza=(prediccion-mina)/(maxa-mina)*(nuevomax-nuevomin)+nuevomin
            certeza=str(int((certeza)*100))+'%'
        marcas.append(marca)
        certezas.append(certeza)
    return prediccion, marcas, certezas


In [4]:
def predecirNuevoCliente_NN(age=40, job='management', marital='married', education='tertiary',
                            balance=5000, housing=0, loan=0, contact='cellular',duration='60', campaign=1):
    cnames=['age', 'job','marital', 'education',
                            'balance', 'housing', 'loan', 'contact','duration', 'campaign']
    Xnew=[age,job,marital,education,balance,housing,loan,contact,duration,campaign]
    Xnew_Dataframe = pd.DataFrame(data=[Xnew],columns=cnames)
    pipe=cargarPipeline("pipeline_bank_data")
    Xnew_Transformado=pipe.transform(Xnew_Dataframe)
    modelo=cargarModelo("model_NN_bank_data")

    y_pred=modelo.predict(Xnew_Transformado)
    predicciones, marcas, certezas= obtenerResultadosyCertezas(y_pred)
    dataframeFinal=pd.DataFrame({'Predicción':predicciones, 'Resultado':marcas, 'Certeza': certezas})
    np.set_printoptions(formatter={'float': lambda X: "{0:0.0f}".format(X)})
    return dataframeFinal

In [5]:
dataframeFinal=predecirNuevoCliente_NN()
dataframeFinal.head()

1/1 [==============================] - 0s 171ms/step


,Predicción,Resultado,Certeza
0,0.005003,No Acepta,98%


Nuevas predicciones (Red Neuronal)

In [14]:
dataframe_prediccion=predecirNuevoCliente_NN(age=30, job='admin.', marital='single', education='tertiary',
                            balance=15000, housing=0, loan=0, contact='cellular',duration='200', campaign=1)
dataframe_prediccion.head()


1/1 [==============================] - 0s 80ms/step


,Predicción,Resultado,Certeza
0,0.00001,No Acepta,99%


In [7]:
dataframe_prediccion=predecirNuevoCliente_NN(age=70, job='retired', marital='married', education='secondary',
                            balance=30000, housing=0, loan=0, contact='telephone',duration='400', campaign=0)
dataframe_prediccion.head()

1/1 [==============================] - 0s 84ms/step


,Predicción,Resultado,Certeza
0,0.077675,No Acepta,84%


### Prediccion con Naive Bayes



In [8]:
# Funcion para cargar modelo Naivebayes
import pickle

def cargarModeloNaiveBayes(nombre_archivo):
# Cargar el modelo
  with open(nombre_archivo, 'rb') as file:
      modelo_cargado = pickle.load(file)
  return modelo_cargado




In [9]:
def predecirNuevoCliente_NB(age=40, job='management', marital='married', education='tertiary',
                            balance=5000, housing=0, loan=0, contact='cellular',duration='60', campaign=1):
    cnames=['age', 'job','marital', 'education',
                            'balance', 'housing', 'loan', 'contact','duration', 'campaign']
    Xnew=[age,job,marital,education,balance,housing,loan,contact,duration,campaign]
    Xnew_Dataframe = pd.DataFrame(data=[Xnew],columns=cnames)
    pipe=cargarPipeline("pipeline_bank_data")
    Xnew_Transformado=pipe.transform(Xnew_Dataframe)
    modelo=cargarModeloNaiveBayes("model_NB_bank_data.pickle")

    y_pred=modelo.predict(Xnew_Transformado)
    predicciones, marcas, certezas= obtenerResultadosyCertezas(y_pred)
    dataframeFinal=pd.DataFrame({'Predicción':predicciones, 'Resultado':marcas, 'Certeza': certezas})
    np.set_printoptions(formatter={'float': lambda X: "{0:0.0f}".format(X)})
    return dataframeFinal

In [10]:
dataframeFinal=predecirNuevoCliente_NB()
dataframeFinal.head()

,Predicción,Resultado,Certeza
0,0,No Acepta,100%


Nuevas predicciones (Naive Bayes)

In [11]:
dataframe_prediccion=predecirNuevoCliente_NB(age=30, job='admin.', marital='single', education='tertiary',
                            balance=15000, housing=0, loan=0, contact='cellular',duration='200', campaign=1)
dataframe_prediccion.head()

,Predicción,Resultado,Certeza
0,1,Acepta,100%


In [12]:
dataframe_prediccion=predecirNuevoCliente_NB(age=70, job='retired', marital='married', education='secondary',
                            balance=30000, housing=0, loan=0, contact='telephone',duration='400', campaign=0)
dataframe_prediccion.head()

,Predicción,Resultado,Certeza
0,1,Acepta,100%


## Conclusiones

Al igual que prácticas anteriores con este dataset, el mejor modelo para realizar predicciones es la Red Neuronal.

Para esta práctica se utilizó el dataset completo el cual constaba con más de 45211 filas. En la fase de análisis exploratorio se encontró que la variable objetivo de este dataset presentaba más 0s que 1s, lo que indicaba un desbalance, en la fase de modelado y entrenamiento de la Red Neuronal lo primero que se trató de hacer fue aplicar las distintas técnicas para tratar el desbalance de datos, como penalización en los pesos, oversampling, subsampling y smote-tomek, pero al aplicarlos no hubo mejora en las medidas de evaluación, entonces sé probo la estrategia de **validación cruzada**, la cual venía bien, ya que nuestro dataset tiene varias filas, y se notó una gran mejora alcanzando medidas de evaluación bastante altas, por encima de 0.8, esta misma estrategia se utilizó para entrenar el modelo de Naive Bayes.

Para el modelo de Naive Bayes se investigó varios tipos de clasificadores, como el Gaussian, Bernoulli, Multinomial, y Complement, los dos últimos modelos no servían en nuestro dataset, ya que al aplicar los transformadores se tenía números negativos (debido a la estandarización) y estos modelos no trabajan con valores negativos, por lo que solo se obtuvo resultados con Gausian y Bernoulli.

Si bien al momento de hacer predicciones, el modelo de Naive Bayes presenta una certeza del 100%, este resultado no es preciso, como se puede ver en la tabla de comparación (tabla 1), esto es debido a que los clasificadores de Naive Bayes devuelven solo 0 o 1, por lo que a la hora de calcular la certeza siempre se obtiene un valor del 100%.




# Fase 6: Desarrollo del Proyecto en el framework de Django

Se tiene un proyecto de Django con la siguiente estructura:

![proyectodjango](https://raw.githubusercontent.com/juandtap/IA_practica_05/master/imagenes/proyecto_django.png)

Se tiene el proyecto con tres "apps", la primera app es ```home``` que contiene la pagina de inicio y se encarga de redirigir a las otras dos aplicaciones.

La segunda app es la prediccion de credito bancario ```app_credito_banco``` la cual es el proyecto que trabajamos como ejemplo.

Por ultimos tenemos nuestra aplicacion de predecir si un cliente del banco se suscribe a un Deposito de plazo fijo ```app_marketing_banco```

En nuestra app ```app_marketing_banco```, usamos los archivos que hemos obtenido en las fases 1, 2 y 3.
* pipeline con los transformadores
* modelo de la red neuronal
* modelo de naive bayes

Despues de agregar plantillas para solicitar el ingreso de datos, tenemos los siguientes resultados:



![proyectodjango1](https://raw.githubusercontent.com/juandtap/IA_practica_05/master/imagenes/proyecto_django_1.png)


![proyectodjango2](https://raw.githubusercontent.com/juandtap/IA_practica_05/master/imagenes/proyecto_django_2.png)

![proyectodjango3](https://raw.githubusercontent.com/juandtap/IA_practica_05/master/imagenes/proyecto_django_3.png)

Enlace al repositorio en git:

https://github.com/juandtap/MachineLearningServices




## Fase 7. Despliegue en un servidor en la nube (Google Cloud)

En google cloud se creo una maquina virtual con ubuntu 20, en la cual se instalaron todas las librerias necesarias para ejecutar el proyecto y se desplego la aplicacion en un servidor web nginx como proxy inverso y con gunicorn como servidor WSGI.

![proyecto_django_6](https://raw.githubusercontent.com/juandtap/IA_practica_05/master/imagenes/proyecto_django_6.png)

![proyecto_django_7](https://raw.githubusercontent.com/juandtap/IA_practica_05/master/imagenes/proyecto_django_7.png)

![proyecto_django_8](https://raw.githubusercontent.com/juandtap/IA_practica_05/master/imagenes/proyecto_django_8.png)

Al ingresar en la direccion IP de la maquina virtual (http://104.196.1.232/) podemos ver nuestra app desplegada.

![proyecto_django_4](https://raw.githubusercontent.com/juandtap/IA_practica_05/master/imagenes/proyecto_django_4.png)

![proyecto_django_5](https://raw.githubusercontent.com/juandtap/IA_practica_05/master/imagenes/proyecto_django_5.png)





## Referencias

1.9. Naive Bayes. (n.d.). Scikit-learn. https://scikit-learn.org/stable/modules/naive_bayes.html

Camacho, J. A., & Camacho, J. A. (2021, August 20). Clasificador Naive Bayes | JacobSoft. JacobSoft. https://www.jacobsoft.com.mx/es_mx/clasificador-naive-bayes/

Na, & Na. (2020b, March 3). Clasificación con datos desbalanceados | Aprende Machine Learning. Aprende Machine Learning. https://www.aprendemachinelearning.com/clasificacion-con-datos-desbalanceados/

Ortiz, I. R. H., PhD. (2023d, December 23). Despliegue de un servicio de machine learning con Flask y Django. Pruebas con Postman - GenSciNet. GenSciNet. https://genscinet.com/despliegue-servicio-machine-learning-flask-django/

